In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from matplotlib import cbook, rc_params_from_file, rcParamsDefault
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score                               
from sklearn import metrics

#classifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
from sklearn import svm

from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv(r"/kaggle/input/food-demand-forecasting/Train.csv")
test = pd.read_csv(r"/kaggle/input/food-demand-forecasting/Test.csv")
fci = pd.read_csv(r"/kaggle/input/food-demand-forecasting/Fulfilment Center Info.csv")
meal = pd.read_csv(r"/kaggle/input/food-demand-forecasting/Meal Info.csv")

In [ ]:
print(train.shape)
train.head()

In [ ]:
train.columns

In [ ]:
train.info()

In [ ]:
print(test.shape)
test.head()

In [ ]:
test.columns

In [ ]:
test.info()

In [ ]:
print(fci.shape)
fci.head()

In [ ]:
fci.columns

In [ ]:
fci.info()

In [ ]:
print(meal.shape)
meal.head()

In [ ]:
meal.columns

In [ ]:
meal.info()

In [ ]:
train = pd.merge(train,fci, on='center_id')
test = pd.merge(test,fci, on='center_id')

train = pd.merge(train,meal, on='meal_id')
test = pd.merge(test,meal, on='meal_id')

train.drop("id", axis = 1, inplace = True)
test.drop("id", axis = 1, inplace = True)

In [ ]:
df = pd.concat([train, test, fci, meal], ignore_index = True, axis = 0).reset_index(drop = True)
print(train.shape, test.shape, fci.shape, meal.shape, df.shape)

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.notnull().tail()

In [ ]:
df.nunique()

In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().sum(axis = 0)

In [ ]:
df.shape

In [ ]:
df.dropna(how = 'any').shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(keep = 'first').shape

In [ ]:
df.drop_duplicates(keep = 'last').shape

In [ ]:
df.drop_duplicates(keep = False).shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.apply(lambda x: x.dtype)

In [ ]:
df.columns.to_series().groupby(df.dtypes).groups

In [ ]:
round((df.apply(lambda x:x.isnull().sum())/len(df))*100,2)

In [ ]:
#Checking for percentage of missing values in each columns
(df.isnull().sum()/len(df))*100

In [ ]:
total_miss = df.isnull().sum()
perc_miss = total_miss/df.isnull().count()*100

missing_data = pd.DataFrame({'Total missing':total_miss,'% missing':perc_miss})

missing_data.sort_values(by='Total missing',ascending=False).head(3)

In [ ]:
print('Unique Values for Each Feature: \n')
for i in df.columns:
    print(i, ':',df[i].nunique()) 

In [ ]:
# find the unique values from categorical features
for col in df.select_dtypes(include='object').columns:
    print(col)
    print(df[col].unique())

In [ ]:
numerical_data = df.select_dtypes(include=np.number) # select_dtypes selects data with numeric features
numerical_col = numerical_data.columns 

print("Numeric Features:")
print(numerical_data.head())
print("===="*20)

In [ ]:
categorical_data = df.select_dtypes(exclude=np.number) # we will exclude data with numeric features
categorical_col = categorical_data.columns                          # we will store the categorical features in a variable

print("Categorical Features:")
print(categorical_data.head())
print("===="*20)

In [ ]:
categorical_features = [feature for feature in df.columns if ((df[feature].dtypes=='O') & (feature not in ['deposit']))]
categorical_features

In [ ]:
### numerical 
numerical_cols = list(df.select_dtypes(exclude=['object']))
numerical_cols

In [ ]:
### categorical
categorical_cols = list(df.select_dtypes(include=['object']))
categorical_cols

In [ ]:
# list of numerical variables
numerical_features = [feature for feature in df.columns if ((df[feature].dtypes != 'O') & (feature not in ['y']))]
print('Number of numerical variables: ', len(numerical_features))

In [ ]:
#Discrete Numerical Features
discrete_feature=[feature for feature in numerical_features if len(df[feature].unique())<25]
print("Discrete Variables Count: {}".format(len(discrete_feature)))

In [ ]:
#Continuous Numerical Features
continuous_features=[feature for feature in numerical_features if feature not in discrete_feature+['deposit']]
print("Continuous feature Count: {}".format(len(continuous_features)))

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
weekly_orders = df.groupby('week',as_index=False)['num_orders'].sum()
weekly_orders.rename(columns={'num_orders':'total_orders'},inplace=True)
weekly_orders.tail()

In [ ]:
df.hist(figsize=(20,20))
plt.show()

In [ ]:
matrix = df.corr() 
f, ax = plt.subplots(figsize=(25, 12)) 
sns.heatmap(matrix, vmax=.8, square=True, cmap="RdYlGn",annot = True);

In [ ]:
sns.pairplot(df)

In [ ]:
def bar_plot(variable):
    var = df[variable]
    varValue = var.value_counts()
    plt.figure(figsize=(15,3))
    plt.bar(varValue.index, varValue,color=['#00008b','#00e5ee','#cd1076', '#008080','#cd5555','red','blue'])
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    
    plt.show()
    print("{}: \n {}".format(variable,varValue))

In [ ]:
categorical_cols = ['center_type', 'category', 'cuisine']
for c in categorical_cols:
    bar_plot(c)

In [ ]:
columns = ['center_type', 'category', 'cuisine']
fig = plt.figure(figsize = (30, 20))
fig.subplots_adjust(hspace = 0.4, wspace = 0.4)
for i in range(1, (len(columns)**2)+1):
    ax = fig.add_subplot(len(columns), len(columns), i)
    ax = sns.countplot(df[columns[int((i-1)/len(columns))]],hue = columns[((i-1)%len(columns))],data = df)
    ax = plt.legend(loc = 'best')
plt.show()

In [ ]:
# Creating a new variable for ratio in checkout and base price
df['priceratio'] = df['base_price'] / df['checkout_price']
df.head()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = (15,16))
sns.countplot(df['center_type'], ax = ax[0,0])
sns.countplot(df['category'], ax = ax[0,1])
sns.countplot(df['cuisine'], ax = ax[1,0])

In [ ]:
df_idx = df.set_index('week')
sumbyweek = df_idx['num_orders'].groupby(level=0).sum()
maxbyweek = df_idx['num_orders'].groupby(level=0).max()

In [ ]:
df_idx.head()

In [ ]:
sumbyweek.head()

In [ ]:
maxbyweek.head()

In [ ]:
num_orders_week_agg = pd.DataFrame({'Num_orders_Sum_by_Week': sumbyweek, 'Num_orders_Max_by_Week': maxbyweek})
num_orders_week_agg.head()

In [ ]:
num_orders_week_agg.sort_values(by='Num_orders_Max_by_Week',ascending=False).head()

In [ ]:
meal_base_price = df[['week', 'center_id', 'meal_id', 'base_price']]
meal_base_price = meal_base_price.set_index(['meal_id', 'center_id', 'week'])
meal_base_price = meal_base_price.sort_index()
meal_base_price.head()

In [ ]:
meal_per_center = df[['week', 'center_id', 'meal_id', 'num_orders']]
meal_per_center = meal_per_center.set_index(['meal_id', 'center_id', 'week'])
meal_per_center = meal_per_center.sort_index()
meal_per_center.head()

In [ ]:
meal_across_center = df[['week', 'meal_id', 'num_orders']]
meal_across_center = meal_across_center.set_index(['meal_id', 'week'])
meal_across_center = meal_across_center.sort_index()
meal_across_center.head()

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
print(df.std())

In [ ]:
print("Total Number Of Centers : ", df['center_id'].nunique())

In [ ]:
print("Total Number Of Cities: ", df['city_code'].nunique())

In [ ]:
print("Total Number Of Regions: ", df['region_code'].nunique())

# Deriving New Metrics Ratio Based On Checkout Price And Base Price

In [ ]:
df.loc[df['checkout_price'] < df['base_price'], 'C'] = 1
df.loc[df['checkout_price'] > df['base_price'], 'C'] = 2
df.loc[df['checkout_price'] == df['base_price'], 'C'] = 0

df['ratio']=df['checkout_price']/df['base_price']
df['ratio'] = df['ratio'].apply(lambda x: 1 if(x<0.5) else x)

# Deriving New Metrics Called Category Based On Checkout Price

In [ ]:
df.loc[df['checkout_price'] <= 100, 'Category_check'] = 0
df.loc[(df['checkout_price'] > 100) & (df['checkout_price'] <= 150), 'Category_check'] = 1
df.loc[(df['checkout_price'] > 150) & (df['checkout_price'] <= 200), 'Category_check'] = 2
df.loc[(df['checkout_price'] > 200) & (df['checkout_price'] <= 300), 'Category_check'] = 3
df.loc[(df['checkout_price'] > 300) & (df['checkout_price'] <= 500), 'Category_check'] = 4
df.loc[(df['checkout_price'] > 500) , 'Category_check'] = 5

# Deriving New Metrics Called Category Based On Base Price

In [ ]:
df.loc[df['base_price'] <= 200, 'Category_base'] = 0
df.loc[(df['base_price'] > 200) & (df['base_price'] <= 400), 'Category_base'] = 1
df.loc[(df['base_price'] > 400) & (df['base_price'] <= 500), 'Category_base'] = 2
df.loc[(df['base_price'] > 500) , 'Category_base'] = 3

In [ ]:
df.head()

# Univariate Analysis

In [ ]:
df['meal_id'].value_counts()

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(y = df['meal_id'])

In [ ]:
df['emailer_for_promotion'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['emailer_for_promotion'])

In [ ]:
plt.figure(figsize=(10,5))
df['emailer_for_promotion'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['homepage_featured'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['homepage_featured'])

In [ ]:
plt.figure(figsize=(10,5))
df['homepage_featured'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['city_code'].value_counts()

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(y = df['city_code'])

In [ ]:
plt.figure(figsize=(10,10))
df['city_code'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['region_code'].value_counts()

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(y = df['region_code'])

In [ ]:
plt.figure(figsize=(10,10))
df['region_code'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['center_type'].value_counts()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y = df['center_type'])

In [ ]:
plt.figure(figsize=(10,10))
df['center_type'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['op_area'].value_counts()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y = df['op_area'])

In [ ]:
plt.figure(figsize=(10,10))
df['op_area'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['category'].value_counts()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y = df['category'])

In [ ]:
plt.figure(figsize=(10,10))
df['category'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['cuisine'].value_counts()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y = df['cuisine'])

In [ ]:
plt.figure(figsize=(10,10))
df['cuisine'].value_counts().plot.pie(autopct="%0.2f%%")

# What is the trend in Meal?

In [ ]:
cuisine_trend = df.pivot_table(index = 'week', columns = 'cuisine', values = 'num_orders', aggfunc = 'mean')

cuisine_trend.plot(figsize = (15, 10))
plt.title("Meal cuisine Trend")
plt.xlabel("Weeks")
plt.ylabel("Orders")
plt.show()

**Below points can be concluded from above plot:**

1. Customers really likes Italian meals
2. Continental meals have consistently lower orders as compare to other cuisine
3. There is fluctuations in Indian meals

In [ ]:
category_trend = df.pivot_table(index = 'week', columns = 'category', values = 'num_orders', aggfunc = 'mean')
category_trend = category_trend.fillna(0)

category_trend.plot(figsize = (15, 10))
plt.title("Meal category Trend")
plt.xlabel("Weeks")
plt.ylabel("Orders")

plt.show()

**Below points can be concluded from above plot:**

1. Customers mostly orders Beverages
2. Many categories, like Soup, Pizza and Fish, with low orders
3. There is fluctuations in Sandwich and Rice Bowl
4. After week 18, Salad became popular with customers.

# What is the trend in Centers ?

In [ ]:
center_type_trend = df.pivot_table(index = 'week', columns = 'center_type', values = 'num_orders', aggfunc = 'mean')

center_type_trend.plot(figsize = (15, 10))
plt.title("Center Type Trend")
plt.xlabel("Weeks")
plt.ylabel("Orders")
plt.show()

**Center type TYPE_B always has more orders than other 2 types**

In [ ]:
region_code_trend = df.pivot_table(index = 'week', columns = 'region_code', values = 'num_orders', 
                                        aggfunc = 'mean')

region_code_trend.plot(figsize = (15, 10))
plt.title("Region code Trend")
plt.xlabel("Weeks")
plt.ylabel("Orders")
plt.show()

**Below points can be concluded from above plot:**

1. Region code 35 has lowest orders
2. There are fluctuations in almost all regions

In [ ]:
op_area_trend = df.copy()
op_area_trend['op_area'] = np.floor(op_area_trend['op_area'])
op_area_trend = op_area_trend.pivot_table(index = 'week', columns = 'op_area', values = 'num_orders', aggfunc = 'mean')

op_area_trend.plot(figsize = (15, 10))
plt.title("Operation Area Trend")
plt.xlabel("Weeks")
plt.ylabel("Orders")
plt.show()

**Orders has positive correlation with operating area**

In [ ]:
df['category'] = df['category'].map({'Beverages':0, 'Rice Bowl':1, 'Starters':2, 'Pasta':3, 'Sandwich':4, 'Biryani':5, 'Extras':6, 
                                     'Pizza':7, 'Seafood':8, 'Other Snacks':9, 'Desert':10, 'Salad':11, 'Fish':12, 'Soup':13})
df.head()

In [ ]:
df['center_type'] = df['center_type'].map({'TYPE_A':0, 'TYPE_B':1, 'TYPE_C':2})
df.head()

In [ ]:
df['cuisine'] = df['cuisine'].map({'Thai':0, 'Indian':1, 'Italian':2, 'Continental':3})
df.head()

# Dist Plot

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df['num_orders'])

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df['city_code'])

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df['op_area'])

# Joint Plot

In [ ]:
plt.figure(figsize = (10, 5))
sns.jointplot(x = 'checkout_price',y = 'num_orders',data = df)

In [ ]:
plt.figure(figsize = (10, 5))
sns.jointplot(x = 'op_area',y = 'num_orders',data = df)

In [ ]:
plt.figure(figsize = (10, 5))
sns.jointplot(x = 'region_code',y = 'num_orders',data = df)

In [ ]:
plt.figure(figsize = (10, 5))
sns.jointplot(x = 'city_code',y = 'num_orders',data = df)

In [ ]:
plt.figure(figsize = (10, 5))
sns.jointplot(x = 'center_id',y = 'num_orders',data = df)

# Bar Plot

In [ ]:
plt.figure(figsize = (25,15))
sns.barplot(df['center_id'],df['num_orders'])

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(df['emailer_for_promotion'],df['num_orders'])

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(df['homepage_featured'],df['num_orders'])

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(df['region_code'],df['num_orders'])

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(df['center_type'],df['num_orders'])

In [ ]:
plt.figure(figsize = (25,15))
sns.barplot(df['op_area'],df['num_orders'])

In [ ]:
plt.figure(figsize = (20,7))
sns.barplot(df['category'], df['num_orders'])

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(df['cuisine'],df['num_orders'])

# Box Plot

In [ ]:
sns.boxplot(x = df['week'])

In [ ]:
sns.boxplot(x = df['center_id'])

In [ ]:
sns.boxplot(x = df['meal_id'])

In [ ]:
sns.boxplot(x = df['checkout_price'])

In [ ]:
sns.boxplot(x = df['base_price'])

In [ ]:
sns.boxplot(x = df['num_orders'])

In [ ]:
sns.boxplot(x = df['city_code'])

In [ ]:
sns.boxplot(x = df['region_code'])

In [ ]:
sns.boxplot(x = df['op_area'])

# Count Plot

In [ ]:
plt.figure(figsize = (10, 10))
sns.countplot(x = "center_type", hue = "cuisine", data = df)

In [ ]:
plt.figure(figsize = (15, 10))
sns.countplot(x = "category", hue = "cuisine", data = df)

In [ ]:
plt.figure(figsize = (15, 10))
sns.countplot(x = "op_area", hue = "cuisine", data = df)

# Violin Plot

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "center_type", y = "op_area", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "cuisine", y = "category", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "center_type", y = "category", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "center_type", y = "cuisine", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "center_type", y = "region_code", data = df)

In [ ]:
plt.figure(figsize = (10, 7))
ax = sns.violinplot(x = "center_type", y = "city_code", data = df)

# Scatter Plot

In [ ]:
plt.figure(figsize = (15, 10))
plt.scatter(df['base_price'], df['num_orders'], label = 'base price')
plt.scatter(df['checkout_price'], df['num_orders'], label = 'checkout price')
plt.ylabel("Number of orders")
plt.xlabel("Price")
plt.show()

In [ ]:
plt.figure(figsize = (15, 10))
plt.scatter(df['base_price'], df['checkout_price'])
plt.title("Base Price vs Checkout Price")
plt.xlabel("Base Price")
plt.ylabel("Checkout Price")
plt.show()

# Point Plot

In [ ]:
fig=plt.figure(figsize = (20,7))
sns.set_style("whitegrid")
plt.title('Pattern of Orders',fontdict={'fontsize':14})

sns.pointplot(x = df.groupby('week').sum().reset_index()['week'],y = df.groupby('week').sum().reset_index()['num_orders'],color = 'coral')

plt.ylabel('No. of Orders',fontdict = {'fontsize':12})
plt.xlabel('Week',fontdict = {'fontsize':12})
sns.despine(bottom = True, left = True);

In [ ]:
fig=plt.figure(figsize = (20,7))
sns.set_style("whitegrid")
plt.title('Pattern of Orders',fontdict={'fontsize':14})

sns.pointplot(x = df.groupby('category').sum().reset_index()['category'],y = df.groupby('category').sum().reset_index()['cuisine'],color = 'coral')

plt.ylabel('Cuisine',fontdict = {'fontsize':12})
plt.xlabel('Category',fontdict = {'fontsize':12})
sns.despine(bottom = True, left = True);

In [ ]:
fig=plt.figure(figsize = (20,7))
sns.set_style("whitegrid")
plt.title('Pattern of Orders',fontdict={'fontsize':14})

sns.pointplot(x = df.groupby('city_code').sum().reset_index()['city_code'],y = df.groupby('city_code').sum().reset_index()['region_code'],color = 'coral')

plt.ylabel('Region Code',fontdict = {'fontsize':12})
plt.xlabel('City Code',fontdict = {'fontsize':12})
sns.despine(bottom = True, left = True);

In [ ]:
fig=plt.figure(figsize = (20,7))
sns.set_style("whitegrid")
plt.title('Pattern of Orders',fontdict={'fontsize':14})

sns.pointplot(x = df.groupby('city_code').sum().reset_index()['city_code'],y = df.groupby('city_code').sum().reset_index()['center_type'],color = 'coral')

plt.ylabel('Center Type',fontdict = {'fontsize':12})
plt.xlabel('City Code',fontdict = {'fontsize':12})
sns.despine(bottom = True, left = True);

In [ ]:
fig=plt.figure(figsize = (20,7))
sns.set_style("whitegrid")
plt.title('Pattern of Orders',fontdict={'fontsize':14})

sns.pointplot(x = df.groupby('region_code').sum().reset_index()['region_code'],y = df.groupby('region_code').sum().reset_index()['center_type'],color = 'coral')

plt.ylabel('Center Type',fontdict = {'fontsize':12})
plt.xlabel('Region Code',fontdict = {'fontsize':12})
sns.despine(bottom = True, left = True);

In [ ]:
fig=plt.figure(figsize = (20,7))
sns.set_style("whitegrid")
plt.title('Pattern of Orders',fontdict={'fontsize':14})

sns.pointplot(x = df.groupby('op_area').sum().reset_index()['op_area'],y = df.groupby('op_area').sum().reset_index()['center_type'],color = 'coral')

plt.ylabel('Center Type',fontdict = {'fontsize':12})
plt.xlabel('op_area',fontdict = {'fontsize':12})
sns.despine(bottom = True, left = True);

In [ ]:
fig=plt.figure(figsize = (20,7))
sns.set_style("whitegrid")
plt.title('Pattern of Orders',fontdict={'fontsize':14})

sns.pointplot(x = df.groupby('cuisine').sum().reset_index()['cuisine'],y = df.groupby('cuisine').sum().reset_index()['num_orders'],color = 'coral')

plt.ylabel('Number Of Orders',fontdict = {'fontsize':12})
plt.xlabel('Cuisine',fontdict = {'fontsize':12})
sns.despine(bottom = True, left = True);

In [ ]:
fig=plt.figure(figsize = (20,7))
sns.set_style("whitegrid")
plt.title('Pattern of Orders',fontdict={'fontsize':14})

sns.pointplot(x = df.groupby('category').sum().reset_index()['category'],y = df.groupby('category').sum().reset_index()['num_orders'],color = 'coral')

plt.ylabel('Number Of Orders',fontdict = {'fontsize':12})
plt.xlabel('Category',fontdict = {'fontsize':12})
sns.despine(bottom = True, left = True);

# Cross Tab

In [ ]:
pd.crosstab(df['num_orders'],df['homepage_featured']).style.background_gradient(cmap = 'winter')

In [ ]:
pd.crosstab(df['num_orders'],df['emailer_for_promotion']).style.background_gradient(cmap = 'spring')

In [ ]:
pd.crosstab(df['num_orders'],df['center_type']).style.background_gradient(cmap = 'autumn')

In [ ]:
pd.crosstab(df['category'],df['cuisine']).style.background_gradient(cmap = 'cool')

In [ ]:
pd.crosstab(df['city_code'],df['center_type']).style.background_gradient(cmap = 'Wistia')

In [ ]:
pd.crosstab(df['region_code'],df['center_type']).style.background_gradient(cmap = 'bwr')

In [ ]:
pd.crosstab(df['city_code'],df['region_code']).style.background_gradient(cmap = 'seismic')

In [ ]:
pd.crosstab(df['op_area'],df['center_type']).style.background_gradient(cmap = 'PuOr')

In [ ]:
pd.crosstab(df['city_code'],df['op_area']).style.background_gradient(cmap = 'YlGnBu')

In [ ]:
pd.crosstab(df['center_type'],df['op_area']).style.background_gradient(cmap = 'rocket_r')

# Line Plot

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(df['city_code'],df['num_orders'])

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(df['category'],df['num_orders'])

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(df['week'],df['num_orders'])

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(df['category'],df['num_orders'])

In [ ]:
plt.figure(figsize = (20,7))
sns.lineplot(df['op_area'],df['center_type'])

In [ ]:
plt.figure(figsize=(15,10))
sns.lineplot(x = df['category'], y = df['num_orders'],marker = '*', linestyle = '--', color = 'red')

In [ ]:
plt.figure(figsize=(20,7))
sns.lineplot(x = df['week'], y = df['num_orders'],marker = '*', linestyle = '--', color = 'red')

In [ ]:
plt.figure(figsize=(20,7))
sns.lineplot(x = df['city_code'], y = df['num_orders'],marker = '*', linestyle = '--', color = 'red')

**Removing All NaN Values**

In [ ]:
df = df.dropna()
#print(df)
df.head()

In [ ]:
df.dropna(inplace = True)
df.isnull().sum()

# Derving New Metric Called Month, Year, Quarter From The Week

In [ ]:
df['Month'] = df['week'].apply(lambda x: int(x / 4)+1)
df['Year'] = df['week'].apply(lambda x: int(x / 52)+1)
df['Quarter'] = df['week'].apply(lambda x: int(x / 13)+1)

In [ ]:
df.head()

In [ ]:
df.columns

# Number of Orders Based on Month

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(df["Month"],df["num_orders"])

In [ ]:
plt.figure(figsize=(20,7))
sns.lineplot(df["week"],df["num_orders"])

In [ ]:
plt.figure(figsize=(20,7))
sns.lineplot(df["Quarter"],df["num_orders"])

# Week Wise Analysis Based On Month

In [ ]:
from IPython.html import widgets
from IPython.display import display
from ipywidgets import widgets, interactive
month = widgets.Dropdown(
    options=['All'] + list(df['Month'].unique()),
    value='All',
    description='Month:',
)
def plot(month):
    
    if month=="All":
        plt.figure(figsize=(20,15))
        sns.lineplot(df["week"],df["num_orders"],hue = df["Month"],palette="Set1")
    else:
        p=data[df["Month"]==month]
        plt.figure(figsize=(20,15))
        sns.lineplot(p["week"],p["num_orders"],color="green",label="Month"+str(month))
interactive(plot,month=month)

# Month Wise Analysis Based On Year

In [ ]:
year = widgets.Dropdown(
    options = ['All'] + list(df['Year'].unique()),
    value='All',
    description='Year:',
)
def plot(year):
    if year=="All":
        plt.figure(figsize=(20,15))
        plt.grid(True)
        sns.lineplot(df["Month"],df["num_orders"],hue = df["Year"],style = df["Year"],palette = "Set1")
    else:
        p=data[df["Year"]==year]
        plt.figure(figsize=(20,15))
        sns.lineplot(p["Month"],p["num_orders"],color="green",label="Year"+str(year))
interactive(plot,year=year)

**Regression Algorithms** : 
Regression AlgorithmsRegression is concerned with modeling the relationship between variables that is iteratively refined using a measure of error in the predictions made by the model.

Regression methods are a workhorse of statistics and have been co-opted into statistical machine learning. This may be confusing because we can use regression to refer to the class of problem and the class of algorithm. Really, regression is a process.

The most popular regression algorithms are:

1. Ordinary Least Squares Regression (OLSR)
2. Linear Regression
3. Logistic Regression
4. Stepwise Regression
5. Multivariate Adaptive Regression Splines (MARS)
6. Locally Estimated Scatterplot Smoothing (LOESS)

**Instance-based Algorithms** : 
Instance-based AlgorithmsInstance-based learning model is a decision problem with instances or examples of training data that are deemed important or required to the model.

Such methods typically build up a database of example data and compare new data to the database using a similarity measure in order to find the best match and make a prediction. For this reason, instance-based methods are also called winner-take-all methods and memory-based learning. Focus is put on the representation of the stored instances and similarity measures used between instances.

The most popular instance-based algorithms are:

1. k-Nearest Neighbor (kNN)
2. Learning Vector Quantization (LVQ)
3. Self-Organizing Map (SOM)
4. Locally Weighted Learning (LWL)
5. Support Vector Machines (SVM)

**Regularization Algorithms** : 
Regularization AlgorithmsAn extension made to another method (typically regression methods) that penalizes models based on their complexity, favoring simpler models that are also better at generalizing.

I have listed regularization algorithms separately here because they are popular, powerful and generally simple modifications made to other methods.

The most popular regularization algorithms are:

1. Ridge Regression
2. Least Absolute Shrinkage and Selection Operator (LASSO)
3. Elastic Net
4. Least-Angle Regression (LARS)

**Decision Tree Algorithms** : 
Decision Tree AlgorithmsDecision tree methods construct a model of decisions made based on actual values of attributes in the data.

Decisions fork in tree structures until a prediction decision is made for a given record. Decision trees are trained on data for classification and regression problems. Decision trees are often fast and accurate and a big favorite in machine learning.

The most popular decision tree algorithms are:

1. Classification and Regression Tree (CART)
2. Iterative Dichotomiser 3 (ID3)
3. C4.5 and C5.0 (different versions of a powerful approach)
4. Chi-squared Automatic Interaction Detection (CHAID)
5. Decision Stump
6. M5
7. Conditional Decision Trees

**Bayesian Algorithms** : 
Bayesian AlgorithmsBayesian methods are those that explicitly apply Bayes’ Theorem for problems such as classification and regression.

The most popular Bayesian algorithms are:

1. Naive Bayes
2. Gaussian Naive Bayes
3. Multinomial Naive Bayes
4. Averaged One-Dependence Estimators (AODE)
5. Bayesian Belief Network (BBN)
6. Bayesian Network (BN)

**Clustering Algorithms** : 
Clustering AlgorithmsClustering, like regression, describes the class of problem and the class of methods.

Clustering methods are typically organized by the modeling approaches such as centroid-based and hierarchal. All methods are concerned with using the inherent structures in the data to best organize the data into groups of maximum commonality.

The most popular clustering algorithms are:

1. k-Means
2. k-Medians
3. Expectation Maximisation (EM)
4. Hierarchical Clustering

**Association Rule Learning Algorithms** : 
Assoication Rule Learning AlgorithmsAssociation rule learning methods extract rules that best explain observed relationships between variables in data.

These rules can discover important and commercially useful associations in large multidimensional datasets that can be exploited by an organization.

The most popular association rule learning algorithms are:

1. Apriori algorithm
2. Eclat algorithm

# Training And Testing Data

In [ ]:
X = df.iloc[:, 0].values.reshape(-1, 1)
Y = df.iloc[:, 1].values.reshape(-1, 1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42, shuffle = True)

In [ ]:
st_x = StandardScaler()    
X_train = st_x.fit_transform(X_train)    
X_test = st_x.transform(X_test)  

# Linear Regression

In [ ]:
regressor = LinearRegression()  
regressor.fit(X_train, Y_train) #training the algorithm
#To retrieve the intercept:
print(regressor.intercept_)

#For retrieving the slope:
print(regressor.coef_)

In [ ]:
Y_pred = regressor.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, Y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, Y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

In [ ]:
# Model initialization
regression_model = LinearRegression()
# Fit the data(train the model)
regression_model.fit(X, Y)
# Predict
Y_pred = regression_model.predict(X)

# model evaluation
rmse = mean_squared_error(Y, Y_pred)
r2 = r2_score(Y, Y_pred)

# printing values
print('Slope:' ,regression_model.coef_)
print('Intercept:', regression_model.intercept_)
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)

In [ ]:
import statsmodels.api as sm

X = np.random.rand(100)
Y = X + np.random.rand(100)*0.1

results = sm.OLS(Y,sm.add_constant(X)).fit()

print(results.summary())

plt.scatter(X,Y)

X_plot = np.linspace(0,1,100)
plt.plot(X_plot, X_plot*results.params[0] + results.params[1])

plt.show()

# Random Forest Classifier Model

In [ ]:
#RANDOM FOREST MODEL
random_classifier = RandomForestClassifier(n_estimators=5, random_state=0)
random_classifier.fit(X_train, Y_train)
random_prediction = random_classifier.predict(X_test)

In [ ]:
#Model Accuracy
print(confusion_matrix(Y_test, random_prediction))
print(classification_report(Y_test, random_prediction))
print('Model Accuracy: ',accuracy_score(Y_test, random_prediction))

In [ ]:
# create and fit RandomForestClassifier model  
rfc=RandomForestClassifier()
rfc.fit(X_train, Y_train)

In [ ]:
#predict
pred = rfc.predict(X_test)
pred

In [ ]:
rfc_acc= accuracy_score(Y_test, pred)
print('The accuracy score using the RandomForestClassifier (befor resample) is :',rfc_acc)

In [ ]:
print(classification_report(Y_test, pred))

# Decision Tree Classifier Model

In [ ]:
# create and fit DecisionTreeClassifier model
dtc = DecisionTreeClassifier()
dtc.fit(X_train,Y_train)

In [ ]:
#predict
pred = dtc.predict(X_test)
pred

In [ ]:
dtc_acc = accuracy_score(Y_test, pred)
print('The accuracy score with using the decision tree classifier is :',dtc_acc)

In [ ]:
print(classification_report(Y_test, pred))

# K Nearnest NeighborsClassifier Model

In [ ]:
#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
knn.fit(X_train, Y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred))

In [ ]:
#KNN accuracy score

Knn_acc= accuracy_score(Y_test, knn.predict(X_test))
print('The accuracy socre using the KNeighborsClassifier is :',Knn_acc)